# DataJoint Element for Miniscope Calcium Imaging

#### Open source data pipeline for managing and analyzing functional calcium imaging data acquired with UCLA Miniscopes.

Welcome to the tutorial for the DataJoint Element for Miniscope Calcium Imaging. This
tutorial aims to provide a comprehensive understanding of the open source data pipeline
created using `element-miniscope`.

<drawio diagram here>

The package is designed to seamlessly ingest and track scan-level metadata, image
processing, and curation results across subjects and sessions.

<pipeline diagram here>

By the end of this tutorial you will have a clear grasp on setting up and integrating
`element-miniscope` into your specific research projects and your lab.

**Key Components and Objectives**

+ **Setup**
+ **Activate the DataJoint pipeline.**
1. **Insert session-level metadata.**
2. **Extract scan-level (recording-level) metadata.**
3. **Perform image processing using
   [`CaImAn`](https://github.com/flatironinstitute/CaImAn).**
4. **Visualize the results.**

##### Setup

This tutorial examines data from a UCLA miniscope recording acquired using the Miniscope
DAQ V4. The goal is to extract fluorescence and deconvolved activity traces and use
these traces to study neuronal calcium dynamics and neural activity during
specific behaviors or tasks.

The results of this element can be combined with **other modalities** to create a
complete, customizable data pipeline for your specific lab or study. For instance, you
can combine `element-miniscope` with `element-calcium-imaging` or `element-deeplabcut`
to characterize neural activity along with markerless pose-estimation during behavior.

##### Steps to run `element-miniscope`
The input data for this pipeline is as follows:
+ UCLA Miniscope acquired recording in a dedicated session directory. 
+ The raw data needed to populate this pipeline is contained within the session directory
in the `example_data` folder when running this tutorial in GitHub Codespaces.

In [ ]:
import datajoint as dj
import datetime
import matplotlib.pyplot as plt
import numpy as np

## Combine multiple Elements into a pipeline

Each DataJoint Element is a modular set of tables that can be combined into a complete
pipeline. Here are the definitions for clarity:

+ **Module**: In Python, a module is a file containing definitions and statements. In the context of DataJoint, modules often define and structure related database tables.
+ **Table**: A structured set of data held within the database. It consists of rows and
  columns, much like an Excel spreadsheet.
+ **Schema**: Think of a schema as a container or namespace within the database where related tables are grouped together. It helps organize and manage the database structure.
+ **Element**: A modular set of related tables. 

---

Each Element contains 1 or more modules, and each module declares its own schema in the database.

This tutorial pipeline is assembled from four DataJoint Elements.

| Element | Source Code | Documentation | Description |
| -- | -- | -- | -- |
| Element Lab | [Link](https://github.com/datajoint/element-lab) | [Link](https://datajoint.com/docs/elements/element-lab) | Lab management related information, such as Lab, User, Project, Protocol, Source. |
| Element Animal | [Link](https://github.com/datajoint/element-animal) | [Link](https://datajoint.com/docs/elements/element-animal) | General animal metadata and surgery information. |
| Element Session | [Link](https://github.com/datajoint/element-session) | [Link](https://datajoint.com/docs/elements/element-session) | General information of experimental sessions. |
| Element Miniscope | [Link](https://github.com/datajoint/element-miniscope) | [Link](https://datajoint.com/docs/elements/element-miniscope) |  Miniscope Imaging analysis with CaImAn. |

By importing the modules for the first time, the schemas and tables will be created in
the database.  Once created, importing modules will not create schemas and tables
again, but will allow access to existing schemas/tables.

The Elements are imported and activated within the `tutorial_pipeline` script.

In [ ]:
from tutorial_pipeline import (
    lab,
    subject,
    session,
    miniscope,
)

Each Python module (e.g. `subject`) contains a schema object that enables interaction with the schema in the database.

In [ ]:
subject.schema

The Python classes in the module correspond to a table in the database server.

In [ ]:
subject.Subject()

## Diagram

Let's plot the diagram of tables within multiple schemas and their dependencies using `dj.Diagram()`.

In [ ]:
(dj.Diagram(subject.Subject) + dj.Diagram(session.Session) + dj.Diagram(miniscope))

## Understanding Table Types in DataJoint

In the previous cell, we visualized the relationships between various tables in our pipeline using `dj.Diagram`. As you might have noticed, tables have different colors and shapes. This is because, in DataJoint, tables can be of different types, each serving a unique purpose.

| Table tier | Color and shape | Description | Practical Example |
| -- | -- | -- | -- |
| **Manual table** | Green box | Data entered manually, either by hand or with external helper scripts. | A table containing data about individual subjects, like their birth date or sex. |
| **Lookup table** | Gray box | Small tables containing general, non-changing information or settings. | A table containing available experimental protocols or animal species. |
| **Imported table** | Blue oval | Data automatically ingested but requiring external data. | A table that pulls data from an external file or dataset. |
| **Computed table** | Red circle | Data computed entirely within the pipeline. | A table calculating metrics or statistics from previously stored data. |
| **Part table** | Plain text | Tables associated with a master table, sharing its tier. | A subtable containing specific measurements for each subject in a master subject table. |

### Order matters!

The arrangement of tables in the `dj.Diagram` is not arbitrary. It represents the flow of data and dependencies:

- **Higher-up Tables**: These are typically your starting point. For instance, before you can insert data about an experimental session, you need to have data about the subject of that session.
- **Dependencies**: Tables connected by a line have dependencies. For example, before
  you can populate data about an experiment's results, you need to insert data about the
  experiment setup.

**Quick Check**: Based on the diagram, which tables do you think we would insert data into first?

---

## Interacting with DataJoint Pipelines

DataJoint offers a powerful set of commands that allow us to interact with the pipeline:

- **Insert**: Manually add data to a table.
- **Populate**: Automatically compute and insert data.
- **Query**: Search and filter data.
- **Fetch**: Retrieve data for further analysis or visualization.

In the upcoming sections, we'll get hands-on with these commands. Let's start by
understanding how to manually insert data into our pipeline!

---

### Insert entries into manual tables

Manual tables serve as the foundation upon which our pipeline builds. By entering data
here, we lay the groundwork for subsequent automated analyses.

Let's start with the first table in the schema diagram (i.e. `subject.Subject` table).

To know what data to insert into the table, we can view its dependencies and attributes using the `.describe()` and `.heading` functions.

In [ ]:
print(subject.Subject.describe())

In [ ]:
subject.Subject.heading

The cells above show all attributes of the subject table.
We will insert data into the
`subject.Subject` table. 

In [ ]:
subject.Subject.insert1(
    dict(subject="subject1", subject_birth_date="2023-01-01", sex="U")
)
subject.Subject()

Let's repeat the steps above for the `Session` table and see how the output varies between
`.describe` and `.heading`.

In [ ]:
print(session.Session.describe())

In [ ]:
session.Session.heading

Notice that `describe`, displays the table's structure and highlights its dependencies, such as its reliance on the `Subject` table. These dependencies represent foreign key references, linking data across tables.

On the other hand, `heading` provides an exhaustive list of the table's attributes. This
list includes both the attributes declared in this table and any inherited from upstream
tables.

With this understanding, let's move on to insert a session associated with our subject.

We will insert into the `session.Session` table by passing a dictionary to the `insert1` method.

In [ ]:
session_key = dict(subject="subject1", session_datetime="2023-01-01 00:00:00")

In [ ]:
session.Session.insert1(session_key)
session.Session()

#### Introducing the `SessionDirectory` Table

Every experimental session produces a set of data files. The `SessionDirectory` table's
purpose is to locate these files. It references a directory path relative to a root
directory, defined in `dj.config["custom"]`. More
information about `dj.config` is provided in the [User
Guide](https://datajoint.com/docs/elements/user-guide/).

In [ ]:
session.SessionDirectory.insert1(dict(**session_key, session_dir="session1"))
session.SessionDirectory()

As the workflow diagram indicates, the tables in the `probe` schemas need to
contain data before the tables in the `ephys` schema accept any data. Let's
start by inserting into `probe.Probe`, a table containing metadata about a
multielectrode probe. 

In [ ]:
print(miniscope.Recording.describe)

In [ ]:
miniscope.Recording.heading

The probe metadata is used by the downstream `ProbeInsertion` table which we
insert data into in the cells below:

In [ ]:
lab.Device.heading

In [ ]:
lab.Device.insert1(dict(device="miniscope A", modality="miniscope"))

In [ ]:
miniscope.Recording.insert1(
    dict(
        **session_key,
        recording_id=1,
        device="miniscope A",
        acq_software="Miniscope-DAQ-V4",
    )
)

## Populate

### Automatically populate tables

`miniscope.MiniscopeRecordingInfo` is the first table in this pipeline that can be
automatically populated using using the `populate()` method.

In DataJoint, the `populate()` method is a powerful feature designed to fill tables based on the logic defined in the table's `make` method. Here's a breakdown of its functionality:

- **Automation**: Instead of manually inserting data into each table, which can be error-prone and time-consuming, `populate()` automates the insertion based on the dependencies and relationships already established in the schema.

- **Dependency Resolution**: Before populating a table, `populate()` ensures all its dependencies are populated. This maintains the integrity and consistency of the data.

- **Part Tables**: If a table has part tables associated with it, calling `populate()` on the main table will also populate its part tables. This is especially useful in cases like `ephys.EphysRecording` and its part table `ephys.EphysRecording.EphysFile`, as they are closely linked in terms of data lineage.

- **Restriction**: The `populate()` method can be restricted to specific entries. For instance, by providing a `session_key`, we're ensuring the method only operates on the data relevant to that particular session. This is both efficient and avoids unnecessary operations.

In the upcoming cells, we'll make use of the `populate()` method to fill the `miniscope.MiniscopeRecordingInfo` table and its part table. Remember, while this operation is automated, it's essential to understand the underlying logic to ensure accurate and consistent data entry.


In [ ]:
miniscope.RecordingInfo()

In [ ]:
miniscope.RecordingInfo.File()

In [ ]:
miniscope.RecordingInfo.populate(session_key, display_progress=True)

Let's view the information was entered into each of these tables:

In [ ]:
miniscope.MiniscopeRecordingInfo()

In [ ]:
miniscope.MiniscopeRecordingInfo.File()

We're almost ready to perform image processing with `CaImAn`. An important step before
processing is managing the parameters which will be used in that step. To do so, we will
define the `CaImAn` parameters in a dictionary and insert them into a DataJoint table
`ProcessingParamSet`. This table keeps track of all combinations of your image
processing parameters. You can choose which parameters are used during processing in a
later step.

Let's view the attributes and insert data into `miniscope.ProcessingParamSet`.

In [ ]:
miniscope.ProcessingParamSet.heading

In [ ]:
# insert clustering task manually
params_ks = {
    "fs": 30000,
    "fshigh": 150,
    "minfr_goodchannels": 0.1,
    "Th": [10, 4],
    "lam": 10,
    "AUCsplit": 0.9,
    "minFR": 0.02,
    "momentum": [20, 400],
    "sigmaMask": 30,
    "ThPr": 8,
    "spkTh": -6,
    "reorder": 1,
    "nskip": 25,
    "GPU": 1,
    "Nfilt": 1024,
    "nfilt_factor": 4,
    "ntbuff": 64,
    "whiteningRange": 32,
    "nSkipCov": 25,
    "scaleproc": 200,
    "nPCs": 3,
    "useRAM": 0,
}
ephys.ClusteringParamSet.insert_new_params(
    clustering_method="kilosort2",
    paramset_idx=0,
    params=params_ks,
    paramset_desc="Spike sorting using Kilosort2",
)
ephys.ClusteringParamSet()

Now that we've inserted kilosort parameters into the `ClusteringParamSet` table,
we're almost ready to sort our data. DataJoint uses a `ClusteringTask` table to
manage which `EphysRecording` and `ClusteringParamSet` should be used during processing. 

This table is important for defining several important aspects of
downstream processing. Let's view the attributes to get a better understanding. 

In [ ]:
ephys.ClusteringTask.heading

The `ClusteringTask` table contains two important attributes: 
+ `paramset_idx` 
+ `task_mode` 

The `paramset_idx` attribute tracks
your kilosort parameter sets. You can choose the parameter set using which 
you want spike sort ephys data. For example, `paramset_idx=0` may contain
default parameters for kilosort processing whereas `paramset_idx=1` contains your custom parameters for sorting. This
attribute tells the `Processing` table which set of parameters you are processing in a given `populate()`.

The `task_mode` attribute can be set to either `load` or `trigger`. When set to `load`,
running the processing step initiates a search for exisiting kilosort output files. When set to `trigger`, the
processing step will run kilosort on the raw data. 

In [ ]:
ephys.ClusteringTask.insert1(
    dict(
        session_key,
        insertion_number=1,
        paramset_idx=0,
        task_mode="load",  # load or trigger
        clustering_output_dir="processed/subject5/session1/probe_1/kilosort2-5_1",
    )
)

In [ ]:
ephys.Clustering.populate(session_key, display_progress=True)

While spike sorting is completed in the above step, you can optionally curate
the output of image processing using the `Curation` table. For this demo, we
will simply use the results of the spike sorting output from the `Clustering` task.

In [ ]:
ephys.Curation.heading

In [ ]:
clustering_key = (ephys.ClusteringTask & session_key).fetch1("KEY")
ephys.Curation().create1_from_clustering_task(clustering_key)

Once the `Curation` table receives an entry, we can populate the remaining
tables in the workflow including `CuratedClustering`, `WaveformSet`, and `LFP`. 

In [ ]:
ephys.CuratedClustering.populate(session_key, display_progress=True)
ephys.LFP.populate(session_key, display_progress=True)
ephys.WaveformSet.populate(session_key, display_progress=True)

Now that we've populated the tables in this workflow, there are one of
several next steps. If you have an existing workflow for
aligning waveforms to behavior data or other stimuli, you can easily
invoke `element-event` or define your custom DataJoint tables to extend the
pipeline.

In this tutorial, we will do some exploratory analysis by fetching the data from the database and creating a few plots.

## Querying Data

DataJoint provides a powerful querying system, allowing you to retrieve and work with data stored in your database seamlessly. In this section, we'll explore the fundamental querying concepts.

#### What is a Query?

- A query is essentially a request for data. With DataJoint, you can craft specific queries to fetch data that meets your criteria from the database.

#### The `fetch()` Method

- The primary method for retrieving data from a DataJoint table is `fetch()`.
- **Default Behavior**: Without any arguments, `fetch()` returns a list of dictionaries. Each dictionary corresponds to an entry in the table.
  
#### The `fetch1()` Method

- For tables with a single entry or when you're only interested in the first entry, use `fetch1()`.
- **Default Behavior**: It returns a dictionary of attributes for that one entry.

#### Specific Attributes

- Both `fetch()` and `fetch1()` can be made more specific by providing attributes.
- Example: `fetch1('fps')` will retrieve only the `fps` attribute from the first entry.

#### Restricting Queries

- Often, you don't want to fetch everything. Instead, you might want data related to a specific subject or session.
- DataJoint uses the `&` operator to restrict queries.
- Example: To get all session times for `subject5`, you might use:
  ```python
  subject1_times = (session.Session & "subject = 'subject1'").fetch("session_datetime")
  ```

#### Fetching Primary Keys

- Sometimes, you just need the primary keys of entries.
- Use the `fetch("KEY")` syntax for this. For instance, `(session.Session).fetch("KEY")`.

#### Let's Dive In!

Now that we've established the basics, let's delve deeper into querying with some practical examples.

In [ ]:
lfp_average = (ephys.LFP & "insertion_number = '1'").fetch1("lfp_mean")

In the query above, we fetch a single `lfp_mean` attribute from the `LFP` table.
We also restrict the query to insertion number 1.

Let's go ahead and plot the LFP mean.

In [ ]:
plt.plot(lfp_average)
plt.title("Average LFP Waveform for Insertion 1")
plt.xlabel("Samples")
plt.ylabel("microvolts (uV)")

DataJoint queries are a highly flexible tool to manipulate and visualize your data.
After all, visualizing traces or generating rasters is likely just the start of
your analysis workflow. This can also make the queries seem more complex at
first. However, we'll walk through them slowly to simplify their content in this notebook. 

The examples below perform several operations using DataJoint queries:
- Fetch the primary key attributes of all units that are in `insertion_number=1`.
- Use **multiple restrictions** to fetch timestamps and create a raster plot.
- Use a **join** operation and **multiple restrictions** to fetch a waveform
  trace, along with unit data to create a single waveform plot

In [ ]:
insert_key = (ephys.ProbeInsertion & "insertion_number = '1'").fetch1("KEY")
units, unit_spiketimes = (
    ephys.CuratedClustering.Unit
    & insert_key
    & 'unit IN ("6","7","9","14","15","17","19")'
).fetch("unit", "spike_times")

In [ ]:
x = np.hstack(unit_spiketimes)
y = np.hstack([np.full_like(s, u) for u, s in zip(units, unit_spiketimes)])
plt.plot(x, y, "|")
plt.xlabel("Time (s)")
plt.ylabel("Unit")

In [ ]:
unit_key = (ephys.CuratedClustering.Unit & insert_key & "unit = '15'").fetch1("KEY")
unit_data = (
    ephys.CuratedClustering.Unit * ephys.WaveformSet.PeakWaveform & unit_key
).fetch1()

In [ ]:
unit_data

In [ ]:
sampling_rate = (ephys.EphysRecording & insert_key).fetch1(
    "sampling_rate"
) / 1000  # in kHz
plt.plot(
    np.r_[: unit_data["peak_electrode_waveform"].size] * 1 / sampling_rate,
    unit_data["peak_electrode_waveform"],
)
plt.xlabel("Time (ms)")
plt.ylabel(r"Voltage ($\mu$V)")

## Conclusion

Throughout this notebook, we've used DataJoint to work with database tables and keep
data organized and automate analyses to increase efficiency of data processing. We've
inserted data into tables, used queries to retrieve, manipulate, and visualize ephys data.

Remember, this is just the beginning. As you grow familiar with DataJoint, you'll
uncover even more ways to harness its capabilities for your specific research needs. 

---

To run this tutorial notebook on your own data, please use the following steps:
- Download the mysql-docker image for DataJoint and run the container according to the
  instructions provide in the repository.
- Create a fork of this repository to your GitHub account.
- Clone the repository and open the files using your IDE.
- Add a code cell immediately after the first code cell in the notebook - we will setup
  the local connection using this cell. In this cell, type in the following code. 

```python
import datajoint as dj
dj.config["database.host"] = "localhost"
dj.config["database.user"] = "<your-username>"
dj.config["database.password"] = "<your-password>"
dj.config["custom"] = {"imaging_root_data_dir": "path/to/your/data/dir",
"database_prefix": "<your-username_>"}
dj.config.save_local()
dj.conn()
```

- Run this code block above and proceed with the rest of the notebook.